In [48]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import ast

### Spotify의 wed_API로 데이터 수집
- spotify에서 제공하는 spotipy 라는 라이브러리를 이용

In [67]:
# key_1
# client_credentials_manager = SpotifyClientCredentials(client_id='e48676801c514fd0a6202eb890eaa0f1', client_secret='14d81002649d483f9002f027f575a80d')

# key_2
# client_credentials_manager = SpotifyClientCredentials(client_id='5022af45601e4149a0881fbbe27eef62', client_secret='e8927574ef474d15aec854bcdbb6894d')

# key_3
client_credentials_manager = SpotifyClientCredentials(client_id='f588ee129cec4da4a194ece6210ffab6', client_secret='87a162c57c234a43abb6e604c7da8c2c')

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### 사용 함수

In [3]:
# album_id_search 검색 album_id또는 트랙들id 받기
def album_id_search(albums_id):
    """엘범의 아이디를 최대 20개씩 spotipy를 통해 불러와, 
    엘범 이름과 그 엘범의 수록된 track의 id를 가져온다.

    Args:
        albums_id (STR in list): 엘범의 id를 리스트에

    Returns:
        딕셔너리: {album_name : tracks_id}
    """
    resert = {'album_id': [],
              'album_name': [],
              'album_tracks': []}
    batch_size = 20
    for i in range(0, len(albums_id), batch_size):        
        batch_album_ids = albums_id[i:i+batch_size]
        albums = sp.albums(batch_album_ids)['albums']
        
        for album in albums:
            album_tracks_id = []
            total = album['total_tracks']
            if total > 0: 
                resert['album_name'].append(album['name'])
                resert['album_id'].append(album['id'])
                for track in album['tracks']['items']:
                    album_tracks_id.append(track['id'])
                resert['album_tracks'].append(album_tracks_id)
    return resert

In [4]:
# playlist_id  playlist의 이름과 track_id를 리스트로 받은 후 이름을 key로 dict를 만들어 반환함.

def playlist_id(ids):
    """
    단일 playlist의 id로 검색하며,
    playlist의 name과 track들의 id를 dict로 반환해줍니다.

    Args:
        ids (list): playlist_id list를 입력해야한다.

    Returns:
        dict: 검색된 playlist의 neme이 key, track_id가 value인 dict
    """
    
    id_date = {}
    
    for id in ids:
        id_list = []
        offset = 0
        limit = 100
        while True:
            playlist = sp.playlist_tracks(id, limit=limit, offset=offset)
            tracks = playlist['items']
            for item in tracks:
                track_id = item['track']['id']
                id_list.append(track_id)
        
            
            offset += limit
            if len(tracks) < limit:
                break
             
        playlist_name = sp.playlist(id)['name']
        id_date[playlist_name] = id_list
        
        
    return id_date

In [5]:
# track_id_search track 정보 가져오기

def track_id_search(ids, DF=True, give_artist_id=False, give_album_id=False):
    """track의 id로 트렉의 정보를 받아옵니다.
    설정에 따라 DataFrame으로 받거나 dict로 받을 수 있습니다.
    설정에 따라 artist_id의 리스트를 받을 수 있습니다.
    설정에 따라 album_id의 리스트를 받을 수 있습니다.

    Args:
        ids (list): 트랙의 id를 리스트형태로 입력해야합니다. 각 ID는 STR타입으로 입력해야합니다.
        DF (bool, optional): 데이터를 DataFrame으로 받을지 Dict로 받을지 선택할 수 있습니다. Defaults to True.
        give_artist_id (bool, optional): artist_id_list를 받을 수 있습니다. 중복은 없습니다. Defaults to False.
        give_album_id (bool, optional): artist_id_list를 받을 수 있습니다. 중복은 없습니다. Defaults to False.

    Returns:
        tuple: track의 정보를 DataFrame 객체 또는 사전(dict)을 반환할 수 있습니다. 또 추가적인 조건에 따라 artist_id와 album_id를 결과값으로 반환할 수도 있습니다.
               artist_id가 두번째로, album_id가 세번째로 반환됩니다.
    """
    
    track_data = {'track_id': ids,
                'track_name': [],
                'track_popularity': [],
                'album_id': []}
    
    return_data = []
    batch_size = 20
    
    if give_artist_id == True:
        artist_id_list = []
        for i in range(0, len(ids), batch_size):
            batch_album_ids = ids[i:i+batch_size]
            tracks = sp.tracks(batch_album_ids)['tracks']
            for track in tracks:
                track_data['track_name'].append(track['name'])
                track_data['track_popularity'].append(track['popularity'])
                track_data['album_id'].append(track['album']['id'])
                for j in range(len(track['artists'])):
                    artist_id_list.append(track['artists'][j]['id']) 
                    
        return_data.append(list(set(artist_id_list))) 
                                        
    else:                    
        for i in range(0, len(ids), batch_size):
            batch_album_ids = ids[i:i+batch_size]
            tracks = sp.tracks(batch_album_ids)['tracks']
            for track in tracks:
                track_data['track_name'].append(track['name'])
                track_data['track_popularity'].append(track['popularity'])
                track_data['album_id'].append(track['album']['id'])


    if give_album_id == True:
        return_data.append(list(set(track_data['album_id'])))


    if DF == True:
        track_data = pd.DataFrame(track_data)
    
    return_data.insert(0,track_data)   
    if give_album_id == True or give_artist_id == True:
        return tuple(return_data)
        
    return track_data

In [6]:
# details_to_df 트렉의 특징 받아오기.

def details_to_df(ids):
    """track의 id로 track features를 받아옴 최대 20개씩 받아옴.

    Args:
        ids (list): track의 id를 리스트로 저장해둔 변수

    Returns:
        DataFrame: track features를 DF형태로 반환함.
                   columns = ['track_id', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']
    """
    track_features = []
    batch_size = 50
    for i in range(0, len(ids), batch_size):
        batch_track_ids = ids[i:i + batch_size]
        batch_af = sp.audio_features(batch_track_ids)
        if batch_af is not None:
            track_features.extend([af for af in batch_af if af is not None])
    
    track_details_df = pd.DataFrame(track_features)
    track_details_df = track_details_df[['id', 'danceability', 'energy', 'loudness', 'speechiness',
                                         'acousticness', 'instrumentalness', 'liveness', 'valence',
                                         'tempo', 'duration_ms']]
    track_details_df.columns = ['track_id', 'danceability', 'energy', 'loudness', 'speechiness',
                                'acousticness', 'instrumentalness', 'liveness', 'valence',
                                'tempo', 'duration_ms']
    
    return track_details_df

In [7]:
# search_track_id_year 특정년도 음악 트렉 아이디만

def search_track_id_year(number, year=2021):
    """track_id를 연도기준으로 가져옵니다.

    Args:
        number (int): 가져올 트랙 개수입니다.
        year (int, optional): 연도 설정입니다. Defaults to 2021.

    Returns:
        list: track_id를 리스트로 반환합니다.
    """
    track_id =[]
    for i in range(0,number,50):
        track_results = sp.search(q=f'year:{year}', type='track', limit=50, offset=i)
        for i, t in enumerate(track_results['tracks']['items']):
            track_id.append(t['id'])
    return track_id


In [8]:
# instrumentalness(보컬의 정도), loudness(청각 듣는정도), acousticness(반주구간)
def detail_filt(df):
    details = ['energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'track_popularity']
    for detail in details:
        df = df[df[detail] != 0]
    
    return df

In [9]:
# album_genre 엘범의 이름을 key로 tracks_id를 velue로 가지는 dict를 반환

def album_genre(album_ids, DF=True):
    
    album_data = {}
    batch_size = 20
    
    for i in range(0, len(album_id_setlist), batch_size):
        batch_album_ids = album_id_setlist[i:i+batch_size]
        albums = sp.albums(batch_album_ids)['albums']
        for album in albums:
            album_track_list = []
            album_name = album['name']
            total = album['total_tracks']
            for j in range(total):
                album_track_list.append(album['tracks']['items'][j]['id'])
            album_data[album_name] = album_track_list
                
    
    return album_data

In [10]:
# 플레이리스트 4개에서 음악 전부 가져오기
game_playlist = ['3BGckoXAM2qE5Hb2zWbIiX', '0uBPX5ktAhWU48VSRhTNic', '37i9dQZF1DXdfOcg1fm0VG','06saXsaZA818xjQmA6t9Ee']
playlist_track_id = playlist_id(game_playlist)
playlist_track_id.keys()

dict_keys(['Game Soundtracks (200+ Games)', 'Game Soundtrack', 'Video Game Soundtracks', 'Game Soundtracks'])

In [11]:
print(
len(playlist_track_id['Game Soundtracks (200+ Games)']),
len(playlist_track_id['Game Soundtrack']),
len(playlist_track_id['Video Game Soundtracks']),
len(playlist_track_id['Game Soundtracks']))

2453 401 115 1077


In [12]:
playlist_tracks = list(set(
        playlist_track_id['Game Soundtracks (200+ Games)'] +
        playlist_track_id['Game Soundtrack'] +
        playlist_track_id['Video Game Soundtracks'] +
        playlist_track_id['Game Soundtracks']))

In [13]:
len(playlist_tracks)

3660

In [14]:
# 플레이 리스트에 None값이 존재함 플레이리스트를 불러올때 1곡만 남으면 구조가 바뀌는듯함.
playlist_tracks = [x for x in playlist_tracks if x is not None]

In [15]:
len(playlist_tracks)

3659

In [16]:
playlist_tracks_df = track_id_search(playlist_tracks)
playlist_tracks_df.to_csv('playlist_tracks_df.csv', index=False)

In [17]:
playlist_tracks_df = pd.read_csv('playlist_tracks_df.CSV')

In [45]:
# 트랙의 popularity가 0인 값은 결측치인 값을 확인하여 제거

playlist_tracks_df_filtered = playlist_tracks_df[playlist_tracks_df['track_popularity'] != 0]
print(playlist_tracks_df_filtered.shape[0])
playlist_tracks_df_filtered.head()

#하지만 데이터 수집을 다 하고 결측치 제거를 했어야함.

1896


,track_id,track_name,track_popularity,album_id
1,1jAjzCmdt037k7WzbFXdoU,CastleStorm Theme,7,6BhfTe5WZ6HDBWfCWXhac4
2,7GBP9vqKB9o9wcQrkblICD,One​-​Winged Angel (Final Fantasy VII),50,04uEX0wzsnA5UcaEtxcbal
4,5YFoMXiNvvuxFeToxbEGf6,Sepiks Prime,32,5ZJrV5uHaqQbQNdMIPhSct
6,1rgC5Ac7tyxipNxFFjyOL8,Alone,14,0NY4XVz9uX7a1q6sPgdI4j
7,6dXsS7N0OcYBnqoWHB2el9,A Wintry Paradise (Stranded Ship),14,2GvoGdztP6rTHFFguk1CtW


In [22]:
# 트랙들의 엘범을 가져옴
playlist_tracks_album_list = playlist_tracks_df['album_id'].to_list()
print(len(playlist_tracks_album_list))
playlist_tracks_album_list = list(set(playlist_tracks_album_list))
print(len(playlist_tracks_album_list))

3659
847


In [ ]:
all_track_dict = album_id_search(playlist_tracks_album_list)
all_track_df = pd.DataFrame(all_track_dict)
all_track_df.to_csv('all_track_df.csv', index=False)

In [29]:
print(all_track_df.shape[0])
all_track_df.head()

847


,album_id,album_name,album_tracks
0,0PmpbHFL9NpkXiaOUwdph3,"Overworld (From ""Super Mario Bros. 2"")",[3frIKeDlH0fEhpsC0PiTVI]
1,2mg1zUT1rcdPYJ8RXdPLYD,Super Mario Games - Greatest Themes & Sounds,"[31SnRKErU0WOSMbxaXP1xw, 0vPMwkQ9y5ZsPGVFcCp3K..."
2,3t0h9Jw2lbQFCrBuDUMARd,Deus Ex: Human Revolution (Original Soundtrack),"[50mku7LtsjfuRPy8hNerdB, 2y0YCPYk5cbx0BWEdwaxJ..."
3,2YMWspDGtbDgYULXvVQFM6,Gris (Original Game Soundtrack),"[0UI8Jk6Ps9tHcvIchV1nuO, 3m7ieZFPvfPARawHCs3Mj..."
4,5K6NH2GtheSrgnAa8fgiLZ,Resident Evil 6 Original Soundtrack,"[1zF8MjUDsrXbc6OlKGoHql, 0QW0IQsC5zvyA1Txj8FV6..."


In [33]:
# 확실히 사운드 트랙인 것들만 선택
df_selected = all_track_df[all_track_df['album_name'].str.lower().str.contains('soundtrack|sound track')]
print(df_selected.shape[0])
df_selected.head()

455


,album_id,album_name,album_tracks
2,3t0h9Jw2lbQFCrBuDUMARd,Deus Ex: Human Revolution (Original Soundtrack),"[50mku7LtsjfuRPy8hNerdB, 2y0YCPYk5cbx0BWEdwaxJ..."
3,2YMWspDGtbDgYULXvVQFM6,Gris (Original Game Soundtrack),"[0UI8Jk6Ps9tHcvIchV1nuO, 3m7ieZFPvfPARawHCs3Mj..."
4,5K6NH2GtheSrgnAa8fgiLZ,Resident Evil 6 Original Soundtrack,"[1zF8MjUDsrXbc6OlKGoHql, 0QW0IQsC5zvyA1Txj8FV6..."
5,2gTINDe6o2alMOdH98IQc5,Tom Clancy's Siege (Original Game Soundtrack),"[2S8kkzzHw0CgbIAz0enjdM, 2IYUtqvwxRsYlkBXDJLda..."
7,1XVLOMqM70YmlLnkLCWAR8,Assassin's Creed Valhalla: Out of the North (O...,"[1MZQ61fhvNCKQDxQYdj9lQ, 0XwxvgWzOtArebLFrpk3C..."


In [ ]:
df_selected.to_excel('df_selected.xlsx', index=False)

# 전처리로 맵핑 후 데이터 삽입.

결측치 제거를 안했으면 455개의 엘범을 가져올 수 있었음.

In [38]:
album_selected = pd.read_excel('album_selected.xlsx')
print(album_selected.shape[0])
album_selected.head()

312


,album_id,album_name,genre,vibe,album_tracks
0,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),"플랫포머, 어드벤쳐",귀여운,"['1566sa7tQN3XaET8oWpHB7', '6MLptqfOLeV8uwAqwX..."
1,2wD6ibumlAybToO5Mpy9pK,A Plague Tale: Innocence (Original Soundtrack),"스토리, 어드벤쳐",중세,"['11gkYPiJmBqKyXstgFmgAX', '0ZHPvPGJ9UsrVWlTEc..."
2,5Yzbew7AnCrYDAD0fPkT3T,A Plague Tale: Requiem (Original Soundtrack),"스토리, 어드벤쳐",중세,"['0FtGY03shmZn7uoB0bJrn3', '6NWga8PuAhrLGYZmR9..."
3,6QenoaBO94H2FZ1ys727M4,A Way Out (Original Game Soundtrack),"스토리, 액션",현대,"['1IJ3pSCbzhoaiOhs1VeQ6r', '6IrrD1fMzCuNYV6RhI..."
4,4KAb8KSRVnZqzn0q5LIn62,Age of Empires Definitive Edition (Original So...,전략,중세,"['7BBscCGiaUjMmWWK8Ez1Ut', '1nGohsct7zmYABTvAu..."


In [43]:
# genre더미 DF를 만듬
dummy_genre_df = album_selected['genre'].str.get_dummies(', ')
print(dummy_genre_df.shape[1])
dummy_genre_df.head()

15


,RPG,공포,로그라이크,배틀로얄,생존,슈팅,스토리,스포츠,시뮬레이션,액션,어드벤쳐,잠입,전략,퍼즐,플랫포머
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [42]:
# vibe더미 DF를 만듬
dummy_vibe_df = album_selected['vibe'].str.get_dummies(', ')
print(dummy_vibe_df.shape[1])
dummy_vibe_df.head()

17


,SF,공포,귀여운,근현대,다크판타지,동양,릴랙싱,미스터리,복고풍,신화,아포칼립스,애니메이션,유머,중세,판타지,펑크,현대
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### 두 DF 합치고 기존 컬럼 제거.

In [46]:
df_selected_dummy_genre = pd.concat([album_selected, dummy_genre_df], axis=1).drop('genre', axis=1)
df_selected_dummy_genre = df_selected_dummy_genre.drop('vibe', axis=1)
df_selected_dummy_genre.head()

,album_id,album_name,album_tracks,RPG,공포,로그라이크,배틀로얄,생존,슈팅,스토리,스포츠,시뮬레이션,액션,어드벤쳐,잠입,전략,퍼즐,플랫포머
0,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),"['1566sa7tQN3XaET8oWpHB7', '6MLptqfOLeV8uwAqwX...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,2wD6ibumlAybToO5Mpy9pK,A Plague Tale: Innocence (Original Soundtrack),"['11gkYPiJmBqKyXstgFmgAX', '0ZHPvPGJ9UsrVWlTEc...",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,5Yzbew7AnCrYDAD0fPkT3T,A Plague Tale: Requiem (Original Soundtrack),"['0FtGY03shmZn7uoB0bJrn3', '6NWga8PuAhrLGYZmR9...",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,6QenoaBO94H2FZ1ys727M4,A Way Out (Original Game Soundtrack),"['1IJ3pSCbzhoaiOhs1VeQ6r', '6IrrD1fMzCuNYV6RhI...",0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,4KAb8KSRVnZqzn0q5LIn62,Age of Empires Definitive Edition (Original So...,"['7BBscCGiaUjMmWWK8Ez1Ut', '1nGohsct7zmYABTvAu...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [47]:
df_selected_dummy_vibe = pd.concat([album_selected, dummy_vibe_df], axis=1).drop('genre', axis=1)
df_selected_dummy_vibe = df_selected_dummy_vibe.drop('vibe', axis=1)
df_selected_dummy_vibe.head()

,album_id,album_name,album_tracks,SF,공포,귀여운,근현대,다크판타지,동양,릴랙싱,미스터리,복고풍,신화,아포칼립스,애니메이션,유머,중세,판타지,펑크,현대
0,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),"['1566sa7tQN3XaET8oWpHB7', '6MLptqfOLeV8uwAqwX...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2wD6ibumlAybToO5Mpy9pK,A Plague Tale: Innocence (Original Soundtrack),"['11gkYPiJmBqKyXstgFmgAX', '0ZHPvPGJ9UsrVWlTEc...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,5Yzbew7AnCrYDAD0fPkT3T,A Plague Tale: Requiem (Original Soundtrack),"['0FtGY03shmZn7uoB0bJrn3', '6NWga8PuAhrLGYZmR9...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,6QenoaBO94H2FZ1ys727M4,A Way Out (Original Game Soundtrack),"['1IJ3pSCbzhoaiOhs1VeQ6r', '6IrrD1fMzCuNYV6RhI...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,4KAb8KSRVnZqzn0q5LIn62,Age of Empires Definitive Edition (Original So...,"['7BBscCGiaUjMmWWK8Ez1Ut', '1nGohsct7zmYABTvAu...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### album_tracks 정규화 해주기

In [50]:
album_id_list = album_selected['album_id'].tolist()
album_tracks_list = album_selected['album_tracks'].tolist()

# eval() 함수를 리스트가 통째로 문자열인것을 리스트로 변환
converted_list = [ast.literal_eval(track) for track in album_tracks_list]

In [51]:
# album_id와 track_id를 zip으로 묶어준 후 track_id를 album_id를 정규화함
album_tracks_list = []
for t in zip(album_id_list, converted_list):
    album_tracks_list.append({"album_id": t[0], "album_tracks": t[1]} )
    
album_tracks_df = pd.json_normalize(album_tracks_list, record_path=("album_tracks"), meta=["album_id"])
album_tracks_df = album_tracks_df.rename(columns={0:'album_track'})
album_tracks_df.head()

,album_track,album_id
0,1566sa7tQN3XaET8oWpHB7,2HrVkjNMJVNcjUSWxUzyCR
1,6MLptqfOLeV8uwAqwXaZ1v,2HrVkjNMJVNcjUSWxUzyCR
2,1n4eXR1NWPasD4eTUzu5ke,2HrVkjNMJVNcjUSWxUzyCR
3,0bqn14mmcTK896brFLLprf,2HrVkjNMJVNcjUSWxUzyCR
4,3sPjbQIcFsrH4TlLWZbKo3,2HrVkjNMJVNcjUSWxUzyCR


In [57]:
# 더미와 조인시켜주기
tracks_vibe_df = pd.merge(album_tracks_df, df_selected_dummy_vibe.drop('album_tracks', axis=1), on='album_id', how='left')
tracks_genre_df = pd.merge(album_tracks_df, df_selected_dummy_genre.drop('album_tracks', axis=1), on='album_id', how='left')
tracks_genre_df.head()

,album_track,album_id,album_name,RPG,공포,로그라이크,배틀로얄,생존,슈팅,스토리,스포츠,시뮬레이션,액션,어드벤쳐,잠입,전략,퍼즐,플랫포머
0,1566sa7tQN3XaET8oWpHB7,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,6MLptqfOLeV8uwAqwXaZ1v,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,1n4eXR1NWPasD4eTUzu5ke,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0bqn14mmcTK896brFLLprf,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,3sPjbQIcFsrH4TlLWZbKo3,2HrVkjNMJVNcjUSWxUzyCR,A Hat in Time (Original Game Soundtrack),0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [64]:
album_track_list = tracks_genre_df['album_track'].tolist()
album_track_list

['1566sa7tQN3XaET8oWpHB7',
 '6MLptqfOLeV8uwAqwXaZ1v',
 '1n4eXR1NWPasD4eTUzu5ke',
 '0bqn14mmcTK896brFLLprf',
 '3sPjbQIcFsrH4TlLWZbKo3',
 '7o837RnwjtiJQ0X2JFzWxC',
 '0FtHZFjPoDL6kQGNi4JUOq',
 '5JPwe4PjY5QkfviqTYZgg7',
 '7mSmt2yar5VUXZNBV4AnpE',
 '0pmsfKxP4NB8rituJb7HxT',
 '2jn2wNfVZRKDmpChcAGo37',
 '1uAnBm1hMBWkIFdRwXfCOK',
 '4NjS3fRbr5r7JS5em1hd7F',
 '31cNfeFWpJYJ0rPeLTzXDk',
 '4CBdriBzXUZHpQlcyQ6ODJ',
 '6PGZsRLVpFhHbJLoL2xawa',
 '72sZpOwollMmZxoRB5hEGd',
 '7EloFI1ED8yDiPJWL1Gbjs',
 '0g7S3JquNLgjMyIv7vdcaE',
 '50bJvQSgfHFeZImFfp4QK4',
 '5uODHxCoTuirCnk0oEYyYv',
 '445Tdyge4wC6Qr7wosq5OY',
 '11gkYPiJmBqKyXstgFmgAX',
 '0ZHPvPGJ9UsrVWlTEcvpVn',
 '74kOPy7LnzmsC2N5dJKjDI',
 '0is2WfilJJLuH0vVY66HS9',
 '47GmD2g1aQ8QG5TGS5cxC4',
 '25soVt58KGoy93coV4BMV3',
 '2gVeLf236tEYQ6QPHSPlWl',
 '1jPNVAphBYatS0kInPUBVN',
 '2M7R8NiLoy1yvDwHp6UY8M',
 '0FtGY03shmZn7uoB0bJrn3',
 '6NWga8PuAhrLGYZmR9Zf6N',
 '4ytV0XWAhBD9eZKsFmZcNo',
 '2gjvZ4VRPFXKyW1yYlnafE',
 '6LE78ViqvPs1N1VThP0r8C',
 '5CCS8l2UL9WYU5Mmw7YtN1',
 

In [68]:
album_track_list = tracks_genre_df['album_track'].tolist()
album_track_details_df = details_to_df(album_track_list)

SpotifyException: http status: 400, code:-1 - Unsupported URL / URI., reason: None

In [ ]:
tracks_genre_details_df = pd.merge(tracks_genre_df, album_track_details_df, left_on='album_track', right_on='track_id', how='right')
tracks_genre_details_df2 = pd.merge(tracks_genre_details_df, album_track_name_df[['track_id', 'track_name', 'track_popularity']], on='track_id', how='left')
tracks_genre_details_df2.keys()
desired_order = ['album_track', 'track_name', 'album_id', 'album_name', 'RPG', '공포',
       '로그라이크', '배틀로얄', '생존', '스토리', '스포츠', '슈팅', '시뮬레이션', '액션', '어드벤쳐', '잠입', '전략',
       '퍼즐', '플랫포머', 'track_id', 'danceability', 'energy', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'track_popularity']
tracks_genre_details_df3 = tracks_genre_details_df2.reindex(columns=desired_order)
genre_details = tracks_genre_details_df3.drop('track_id', axis=1)
genre_details.keys()

In [ ]:
tracks_vibe_details_df = pd.merge(tracks_vibe_df, album_track_details_df, left_on='album_track', right_on='track_id', how='right')
tracks_vibe_details_df2 = pd.merge(tracks_vibe_details_df, album_track_name_df[['track_id', 'track_name', 'track_popularity']], on='track_id', how='left')
desired_order = ['album_track','track_name', 'album_id', 'album_name', 'SF', '공포', '귀여운', '근현대', '다크판타지', '동양', '릴랙싱', '미스터리', '복고풍', '신화',
       '아포칼립스', '애니메이션', '유머', '중세', '판타지', '펑크', '현대', 'track_id',
       'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'track_popularity']
tracks_vibe_details_df3 = tracks_vibe_details_df2.reindex(columns=desired_order)
vibe_details = tracks_vibe_details_df3.drop('track_id', axis=1)
vibe_details.keys()